In [1]:
!pip install evidently

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.0
    Uninstalling pydantic-2.6.0:
      Successfully uninstalled pydantic-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not i

In [2]:
import evidently
import time
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently import ColumnMapping
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive

# Montez votre Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Chemin complet vers votre fichier
file_path = '/content/drive/MyDrive/OC-Projet-7/out_put/data_sample.csv'
file_path2 = '/content/drive/MyDrive/OC-Projet-7/out_put/data_test.csv'
# Lisez le fichier avec pandas
import pandas as pd
application_train = pd.read_csv(file_path)
application_test = pd.read_csv(file_path2)

In [ ]:
application_train.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'CODE_GENDER',
       'FLAG_OWN_CAR',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT', 'score'],
      dtype='object', length=804)

In [ ]:
application_test.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'CODE_GENDER',
       'FLAG_OWN_CAR',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MAX',
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT'],
      dtype='object', length=803)

In [ ]:
application_train.drop("score", axis=1, inplace=True)
application_train.shape

(10000, 803)

In [ ]:
application_test.shape

(48744, 803)

In [ ]:
#Pour les colonnes catégorielles on va ne prendre que les colonnes ayant que des 0 et des 1
categorical_columns = []

# Parcourir chaque colonne
for col in application_train.columns:
    # Récupérer les valeurs uniques de la colonne
    unique_vals = set(application_train[col].unique())

    # Vérifier si les valeurs uniques sont uniquement 0, 1, et potentiellement NaN
    if unique_vals.issubset({0, 1, np.nan}):
        categorical_columns.append(col)

numerical_columns = [col for col in application_train.columns if col not in categorical_columns]


# Vérifiez que vos deux DataFrames ont exactement les mêmes colonnes
assert set(application_train.columns) == set(application_test.columns)

# Si l'assertion est réussie, cela signifie que les colonnes correspondent
print("Les colonnes correspondent!")


Les colonnes correspondent!


In [ ]:
# Création du column mapping
column_mapping = ColumnMapping()

column_mapping.numerical_features = numerical_columns
column_mapping.categorical_features = categorical_columns

# Créer le rapport de dérive des données
data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', cat_stattest='psi', num_stattest_threshold=0.1, cat_stattest_threshold=0.2),
])

print("Création du data_drift_report")

Création du data_drift_report


In [ ]:
start_time=time.time()
data_drift_report.run(reference_data=application_train, current_data=application_test, column_mapping=column_mapping)

print("Run du data_drift_report")

elapsed_time_fit = time.time() - start_time
print(elapsed_time_fit)

Run du data_drift_report
61239.55736494064


In [ ]:
# Sauvegardez le rapport en tant que fichier HTML
data_drift_report.save_html('/content/drive/MyDrive/OC-Projet-7/data_drift_report.html')